# Trace模块的使用

python提供了原生的logging模块用于记录各种等级的log。为了便于项目使用，在此用其编写了一个自己的Trace类，用于输出各个模块的log。
本Trace类支持设置单个log文件大小以及log文件个数，每当log达到设定的文件个数后，log会以循环的形式覆盖掉之前的log。

In [12]:
import logging
import logging.handlers

class Trace:
    def __init__(self,modulename='Trace',maxfilecount=5,singlefilesize=2048):
        # maxfilecount：log文件数量，singlefilesize：单个文件大小，当达到这一大小后，将会建立新的log文件
        self.__modulename = modulename # 分模块名来记录log
        self.__logger = logging.getLogger(modulename)
        # 给logger对象设立等级，从低到高为DEBUG,INFO,WARNING,ERROR,CRITICAL，对应的整数是10,20,30,40,50
        self.__logger.setLevel(logging.DEBUG)
        self.__maxfilecount = maxfilecount
        self.__singlefilesize = singlefilesize
        # 设置streamhandler
        self.__streamhandler = logging.StreamHandler()
        self.__logfilename = self.__modulename + '.log'
        # 以RotatingFileHandler作为fileHandler。
        # 对于每个logger对象，可以建立多个handler对象用于处理log内容。
        # StreamHandler用于将log内容输出到屏幕（控制台），而FileHandler用于将log内容存储到指定的文件中。
        # 这里选用RotatingFileHandler来作为logger的FileHandler，可以自动根据文件大小和数量来生成log文件。
        self.__filehandler = logging.handlers.RotatingFileHandler(self.__logfilename,
                                                                  maxBytes=singlefilesize,
                                                                  backupCount=maxfilecount,
                                                                  encoding='utf-8')
        # 设立formatter用于格式化输出log。从左到右依次为时间（到毫秒），模块名，等级，线程ID，具体log信息。
        # logging库本身是线程安全的，因此这里无需过于注意多线程。
        self.__formatter = logging.Formatter('%(asctime)s %(name)s %(levelname)s %(thread)d %(message)s')
        # 给StreamHandler和RotatingFileHandler添加格式化
        self.__streamhandler.setFormatter(self.__formatter)
        # 给logger对象添加上这两个handler
        self.__logger.addHandler(self.__streamhandler)
        self.__filehandler.setFormatter(self.__formatter)
        self.__logger.addHandler(self.__filehandler)

    def trace(self,message,streamlevel=logging.WARNING,filelevel=logging.INFO):
        # 输出log的函数。这里的参数有两个：streamlevel用于设定输出到屏幕的log等级，而filelevel用于设定输出到文件的log等级。
        # 使用handler的目的就在于可以区别输出不同的log。如在这里，屏幕输出的默认只包含WARNING及以上的log，而存入文件的则是包含INFO及以上等级的
        # 给两个handler设置等级
        self.__streamhandler.setLevel(streamlevel)
        self.__filehandler.setLevel(filelevel)
        # 根据等级调用不同的函数输出
        if streamlevel == logging.DEBUG:
            self.__logger.debug(message)
        elif streamlevel == logging.INFO:
            self.__logger.info(message)
        elif streamlevel == logging.WARNING:
            self.__logger.warning(message)
        elif streamlevel == logging.ERROR:
            self.__logger.error(message)
        elif streamlevel == logging.CRITICAL:
            self.__logger.critical(message)
        # 这里用exception来输出各种Exception的信息。
        else:
            self.__logger.exception(message)
        self.__filehandler.close()

可以通过以下的交互界面来试试看。

In [14]:
from ipywidgets import widgets
from IPython.display import display

modulename = widgets.Text(description='模块名称')
maxfilecount = widgets.Text(description='log文件数量')
singlefilesize = widgets.Text(description='单个文件大小')
message = widgets.Text(description='log内容')
streamlevel = widgets.Text(description='屏幕log等级')
filelevel = widgets.Text(description='文件log等级')
button = widgets.Button(description='确定')
display(modulename)
display(maxfilecount)
display(singlefilesize)
display(streamlevel)
display(filelevel)
display(message)
display(button)

def on_button_clicked(b):
    tracer = Trace(modulename.value,int(maxfilecount.value),int(singlefilesize.value))
    tracer.trace(message.value,int(streamlevel.value),int(filelevel.value))
button.on_click(on_button_clicked)
    

Text(value='', description='模块名称')

Text(value='', description='log文件数量')

Text(value='', description='单个文件大小')

Text(value='', description='屏幕log等级')

Text(value='', description='文件log等级')

Text(value='', description='log内容')

Button(description='确定', style=ButtonStyle())

在使用Trace时，我们只需引入Trace类，建立一个Trace对象并设定好文件大小和单个文件数量，就可以通过trace方法来输出不同等级的log了。